In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, TrainingArguments, AutoModelForSequenceClassification, AutoConfig ,AutoModelForCausalLM
import torch

from sklearn.metrics import matthews_corrcoef, f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

/home/lb4489/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_path = "/scratch/lb4489/LLM"

# Import the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", cache_dir=model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", cache_dir=model_path, trust_remote_code=True)

In [4]:
from datasets import load_dataset

ds = load_dataset("BrainGPT/BrainBench_Human_v0.1.csv")

In [9]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['doi', 'journal_section', 'original_abstract', 'incorrect_abstract'],
        num_rows: 200
    })
})


In [29]:
import json
df = ds["train"].to_pandas()


In [43]:
causal_lm_data = [
    {"text": f"错误文本：{row['incorrect_abstract']}\n正确文本：{row['original_abstract']}"}
    for _, row in df.iterrows()
]


In [44]:
print(causal_lm_data)

[{'text': "错误文本：To fluidly engage with the world, our brains must simultaneously represent both the scene in front of us and our memory of the immediate surrounding environment (i.e., local visuospatial context). How does the brain's functional architecture enable sensory and mnemonic representations to closely interface while also avoiding sensory-mnemonic interference? Here, we asked this question using first-person, head-mounted virtual reality and fMRI. Using virtual reality, human participants of both sexes learned a set of immersive, real-world visuospatial environments in which we systematically manipulated the extent of visuospatial context associated with a scene image in memory across three learning conditions, spanning from a single FOV to a city street. We used individualized, within-subject fMRI to determine which brain areas support memory of the visuospatial context associated with a scene during recall (Experiment 1) and recognition (Experiment 2). Across the whole brai

In [46]:
with open("causal_lm_data.json", "w", encoding="utf-8") as f:
    json.dump(causal_lm_data, f, ensure_ascii=False, indent=4)

In [47]:
from sklearn.model_selection import train_test_split

# 读取 JSON 数据
import json

with open("causal_lm_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 拆分数据集
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

with open("train.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

with open("val.json", "w", encoding="utf-8") as f:
    json.dump(val_data, f, ensure_ascii=False, indent=4)


In [48]:
#def preprocess_function(examples):
#    return tokenizer(examples["text"], truncation=True, padding=False, max_length=512)

#from datasets import load_dataset

# 加载拆分后的数据
dataset = load_dataset("json", data_files={"train": "train.json", "validation": "val.json"})

# Tokenization
#tokenized_datasets = dataset.map(preprocess_function, batched=True)


Generating train split: 160 examples [00:00, 6857.36 examples/s]
Generating validation split: 40 examples [00:00, 5592.41 examples/s]
Map: 100%|██████████| 40/40 [00:00<00:00, 1070.90 examples/s]


In [37]:
#pip install unsloth

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from unsloth import FastLanguageModel  
import torch  

#max_seq_length = 2048 
#dtype = None  
#load_in_4bit = True  

#model, tokenizer = FastLanguageModel.from_pretrained(
#    model_name="unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit",  
#    max_seq_length=max_seq_length, 
#    dtype=dtype,  
#    load_in_4bit=load_in_4bit, 
#)

In [38]:
from unsloth.chat_templates import get_chat_template

#tokenizer = get_chat_template(
#    tokenizer,
#    chat_template = "qwen-2.5",
#)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Model does not have a padding token! Will use pad_token = <|vision_pad|>.


In [50]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 160
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 40
    })
})


In [62]:
def format_chatml(example):
    # 直接用 `text` 处理
    parts = example["text"].split("正确文本：")
    if len(parts) != 2:
        return {"text": example["text"]}  # 如果格式不匹配，保持原样

    incorrect_text, correct_text = parts
    return {
        "text": (
            "<|im_start|>user\n"
            f"{incorrect_text.strip()}\n"
            "<|im_start|>assistant\n"
            f"{correct_text.strip()}\n"
            "<|im_end|>"
        )
    }

# 重新格式化 `dataset`
dataset = dataset.map(format_chatml)
print(dataset["train"][0]["text"])

Map: 100%|██████████| 40/40 [00:00<00:00, 3933.97 examples/s]

<|im_start|>user
错误文本：Impulse control and/or gambling disorders can be triggered by dopamine agonist therapies used to treat Parkinson's disease, but the cognitive and neurobiological mechanisms underlying these adverse effects are unknown. Recent data show that adding win-paired sound and light cues to the rat gambling task (rGT) potentiates risky decision-making and impulsivity via the dopamine system, and that changing dopaminergic tone has a greater influence on behavior while subjects are learning task contingencies. Dopamine agonist therapy may therefore be potentiating risk-taking by amplifying the behavioral impact of gambling-related cues on novel behavior. Here, we show that ropinirole treatment in male rats transiently decreased motor impulsivity but robustly and progressively decreased choice of the high-risk/high-reward options when administered during acquisition of the cued but not uncued rGT. Early in training, ropinirole decreased win-stay behavior after large unlikely

In [83]:
from trl import SFTTrainer  
from transformers import TrainingArguments, DataCollatorForSeq2Seq  
from unsloth import is_bfloat16_supported 

max_seq_length = 2048  
#dtype = None 
#load_in_4bit = True 

trainer = SFTTrainer( 
    model=model, 
    tokenizer=tokenizer, 
    train_dataset=dataset["train"], 
    eval_dataset=dataset["validation"],  
    dataset_text_field="text", 
    max_seq_length=max_seq_length, 
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),  
    dataset_num_proc=4,  
    packing=False, 
    args=TrainingArguments( 
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4, 
        warmup_steps=5, 
        max_steps=100, 
        learning_rate=2e-4, 
        fp16=not is_bfloat16_supported(),  
        bf16=is_bfloat16_supported(),
        logging_steps=5, 
        optim="paged_adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear", 
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

Map (num_proc=4): 100%|██████████| 40/40 [00:01<00:00, 28.90 examples/s]
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [84]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)


Map: 100%|██████████| 40/40 [00:00<00:00, 1140.65 examples/s]


In [85]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<｜begin▁of▁sentence｜><|im_start|>user\n错误文本：NG2 is a structurally unique transmembrane chondroitin sulfate proteoglycan (CSPG). Its role in damaged spinal cord is dual. NG2 is considered one of key inhibitory factors restricting axonal growth following spinal injury. Additionally, we have recently detected its novel function as a blocker of axonal conduction. Some studies, however, indicate the importance of NG2 presence in the formation of synaptic contacts. We hypothesized that the optimal treatment would be neutralization of inhibitory functions of NG2 without its physical removal. Acute intraspinal injections of anti-NG2 monoclonal antibodies reportedly prevented an acute block of axonal conduction by exogenous NG2. For prolonged delivery of NG2 function neutralizing antibody, we have developed a novel gene therapy: adeno-associated vector (AAV) construct expressing recombinant single-chain variable fragment anti-NG2 antibody (AAV-NG2Ab). We examined effects of AAV-NG2Ab alone or 

In [86]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                                                                                                                                                                                                                                                                \nNG2 is a structurally unique transmembrane chondroitin sulfate proteoglycan (CSPG). Its role in damaged spinal cord is dual. NG2 is considered one of key inhibitory factors restricting axonal growth following spinal injury. Additionally, we have recently detected its novel function as a blocker of axonal conduction. Some studies, however, indicate the importance of NG2 presence in the formation of synaptic contacts. We hypothesized that the optimal treatment would be neutralization of inhibitory functions of NG2 without its physical removal. Acute intraspinal injections of anti-NG2 monoclon

In [87]:
# 开始训练
trainer_stats = trainer.train()

Step,Training Loss
5,0.039100
10,0.071000
15,0.062300
20,0.081100
25,0.076000
30,0.106800
35,0.135500
40,0.081600
45,0.031500
50,0.046600


In [89]:
trainer.save_model("/scratch/lb4489/LLM/fine_tuned_Qwen")
tokenizer.save_pretrained("/scratch/lb4489/LLM/fine_tuned_Qwen")


('/scratch/lb4489/LLM/fine_tuned_Qwen/tokenizer_config.json',
 '/scratch/lb4489/LLM/fine_tuned_Qwen/special_tokens_map.json',
 '/scratch/lb4489/LLM/fine_tuned_Qwen/tokenizer.json')

In [90]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/scratch/lb4489/LLM"

# Import the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("/scratch/lb4489/LLM/fine_tuned_Qwen", cache_dir=model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("/scratch/lb4489/LLM/fine_tuned_Qwen", cache_dir=model_path, trust_remote_code=True).to("cuda")

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.01it/s]


In [91]:
# 确保 `pad_token` 存在
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 用户输入
user_input = "<|im_start|>user\nNG2 is a structurally unique transmembrane chondroitin sulfate proteoglycan (CSPG). Its role in damaged spinal cord is dual. NG2 is considered one of key inhibitory factors restricting axonal growth following spinal injury. Additionally, we have recently detected its novel function as a blocker of axonal conduction. Some studies, however, indicate the importance of NG2 presence in the formation of synaptic contacts. We hypothesized that the optimal treatment would be neutralization of inhibitory functions of NG2 without its physical removal. Acute intraspinal injections of anti-NG2 monoclonal antibodies reportedly prevented an acute block of axonal conduction by exogenous NG2. For prolonged delivery of NG2 function neutralizing antibody, we have developed a novel gene therapy: adeno-associated vector (AAV) construct expressing recombinant single-chain variable fragment anti-NG2 antibody (AAV-NG2Ab). We examined effects of AAV-NG2Ab alone or in combination with neurotrophin NT-3 in adult female rats with thoracic T10 contusion injuries. A battery of behavioral tests was used to evaluate locomotor function. In vivo single-cell electrophysiology was used to evaluate synaptic transmission. Lower urinary tract function was assessed during the survival period using metabolic chambers. Terminal cystometry, with acquisition of external urethral sphincter activity and bladder pressure, was used to evaluate bladder function. Both the AAV-NG2Ab and AAV-NG2Ab combined with AAV-NT3 treatment groups demonstrated significant improvements in transmission, locomotion, and bladder function compared with the control (AAV-GFP) group. These functional improvements associated with improved remyelination and plasticity of 5-HT fibers. The best results were observed in the group that received combinational AAV-NG2Ab+AAV-NT3 treatment.\n<|im_start|>assistant\n正确文本："

# Tokenization（带 `attention_mask`）
inputs = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True, max_length=1024)
inputs = {key: val.to("cuda") for key, val in inputs.items()}  # 确保 `input_ids` 和 `attention_mask` 在 GPU

# 生成正确文本，避免重复
output_ids = model.generate(
    **inputs,
    max_new_tokens=500,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.2
)

# 解码文本
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 只保留 `<|im_end|>` 之前的内容
if "<|im_end|>" in generated_text:
    generated_text = generated_text.split("<|im_end|>")[0]

print("纠正后的文本：", generated_text)

纠正后的文本： <|im_start|>user
NG2 is a structurally unique transmembrane chondroitin sulfate proteoglycan (CSPG). Its role in damaged spinal cord is dual. NG2 is considered one of key inhibitory factors restricting axonal growth following spinal injury. Additionally, we have recently detected its novel function as a blocker of axonal conduction. Some studies, however, indicate the importance of NG2 presence in the formation of synaptic contacts. We hypothesized that the optimal treatment would be neutralization of inhibitory functions of NG2 without its physical removal. Acute intraspinal injections of anti-NG2 monoclonal antibodies reportedly prevented an acute block of axonal conduction by exogenous NG2. For prolonged delivery of NG2 function neutralizing antibody, we have developed a novel gene therapy: adeno-associated vector (AAV) construct expressing recombinant single-chain variable fragment anti-NG2 antibody (AAV-NG2Ab). We examined effects of AAV-NG2Ab alone or in combination with 

In [93]:
user_input = "<|im_start|>user\n请找出错误并修正，并说明修改原因：\n" \
             "错误文本：Lower urinary tract function was assessed during the survival period using metabolic chambers. " \
             "Terminal cystometry, with acquisition of external urethral sphincter activity and bladder pressure, was used to evaluate bladder function.\n" \
             "<|im_start|>assistant\n"

inputs = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True, max_length=1024)
inputs = {key: val.to("cuda") for key, val in inputs.items()} 

output_ids = model.generate(
    **inputs,
    max_new_tokens=200,  # 不要太长，避免 GPT 生成重复内容
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.2
)

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("GPT 生成的修正文本：", generated_text)

GPT 生成的修正文本： <|im_start|>user
请找出错误并修正，并说明修改原因：
错误文本：Lower urinary tract function was assessed during the survival period using metabolic chambers. Terminal cystometry, with acquisition of external urethral sphincter activity and bladder pressure, was used to evaluate bladder function.
<|im_start|>assistant
 Lower urinary tract function was assessed during the survival period using metabolic chambers. Terminal cystometry, with acquisition of external urethral sphincter activity and bladder pressure, was used to evaluate bladder function.
<|im_end|>user
Lower urinary tract function was assessed during the survival period using metabolic chambers. Terminal cystometry, with acquisition of external urethral sphincter activity and bladder pressure, was used to evaluate bladder function.
<|im_end|>user
Lower urinary tract function was assessed during the survival period using metabolic chambers. Terminal cystometry, with acquisition of external urethral sphincter activity and bladder pressur

In [ ]:
############################
#换一种任务模式

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, TrainingArguments, AutoModelForSequenceClassification, AutoConfig ,AutoModelForCausalLM
import torch

from sklearn.metrics import matthews_corrcoef, f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

/home/lb4489/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "/scratch/lb4489/LLM"

# Import the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", cache_dir=model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", cache_dir=model_path, trust_remote_code=True)

In [3]:
from datasets import load_dataset

ds = load_dataset("BrainGPT/BrainBench_Human_v0.1.csv")

In [4]:
import json
df = ds["train"].to_pandas()

In [5]:
import pandas as pd
import json

def extract_last_sentence(row):
    sentences = row["original_abstract"].strip().split(". ")  # 按句号分割
    if len(sentences) > 1:
        last_sentence = sentences[-1]  # 取最后一句
        rest_of_text = ". ".join(sentences[:-1])  # 剩余部分
        return {
            "input": f"请根据结尾句补全前面的段落：\n最后一句：{last_sentence}.",
            "output": f"完整文本：{rest_of_text}. {last_sentence}."
        }
    else:
        return {
            "input": f"请根据结尾句补全前面的段落：\n最后一句：{row['original_abstract']}.",
            "output": f"完整文本：{row['original_abstract']}."
        }


json_data = df.apply(extract_last_sentence, axis=1).tolist()

json_path = "/scratch/lb4489/LLM/reversed_completion_data.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent=4, ensure_ascii=False)



In [6]:
from sklearn.model_selection import train_test_split

# 读取 JSON 数据
import json

with open("/scratch/lb4489/LLM/reversed_completion_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 拆分数据集
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

with open("train.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

with open("val.json", "w", encoding="utf-8") as f:
    json.dump(val_data, f, ensure_ascii=False, indent=4)

In [7]:
from unsloth import FastLanguageModel  
import torch  
from unsloth.chat_templates import get_chat_template

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [10]:
#def preprocess_function(examples):
#    texts = [f"{i} {o}" for i, o in zip(examples["input"], examples["output"])]  
#    return tokenizer(texts, truncation=True, padding="max_length", max_length=512)


from datasets import load_dataset

# 加载拆分后的数据
dataset = load_dataset("json", data_files={"train": "train.json", "validation": "val.json"})

# Tokenization
#tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 40/40 [00:00<00:00, 1258.79 examples/s]


In [13]:
def format_chatml(example):

    user_input = example.get("input", "").strip()  
    assistant_output = example.get("output", "").strip() 

    if not user_input or not assistant_output:
        return {"text": ""}  # 避免空值

    return {
        "text": (
            "<|im_start|>user\n"
            f"{user_input}\n"
            "<|im_start|>assistant\n"
            f"{assistant_output}\n"
            "<|im_end|>"
        )
    }

dataset = dataset.map(format_chatml, batched=False) 

print(dataset["train"][0]["text"])

Map: 100%|██████████| 40/40 [00:00<00:00, 2977.75 examples/s]

<|im_start|>user
请根据结尾句补全前面的段落：
最后一句：Frontostriatal activity uninformed by the endogenous dopaminergic teaching signal therefore appeared to perpetuate risky choice, and ropinirole exaggerated this disconnect in synergy with reward-paired cues..
<|im_start|>assistant
完整文本：Impulse control and/or gambling disorders can be triggered by dopamine agonist therapies used to treat Parkinson's disease, but the cognitive and neurobiological mechanisms underlying these adverse effects are unknown. Recent data show that adding win-paired sound and light cues to the rat gambling task (rGT) potentiates risky decision-making and impulsivity via the dopamine system, and that changing dopaminergic tone has a greater influence on behavior while subjects are learning task contingencies. Dopamine agonist therapy may therefore be potentiating risk-taking by amplifying the behavioral impact of gambling-related cues on novel behavior. Here, we show that ropinirole treatment in male rats transiently increased

In [14]:
from trl import SFTTrainer  
from transformers import TrainingArguments, DataCollatorForSeq2Seq  
from unsloth import is_bfloat16_supported 

max_seq_length = 2048  
#dtype = None 
#load_in_4bit = True 

trainer = SFTTrainer( 
    model=model, 
    tokenizer=tokenizer, 
    train_dataset=dataset["train"], 
    eval_dataset=dataset["validation"],  
    dataset_text_field="text", 
    max_seq_length=max_seq_length, 
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),  
    dataset_num_proc=4,  
    packing=False, 
    args=TrainingArguments( 
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4, 
        warmup_steps=5, 
        max_steps=100, 
        learning_rate=2e-4, 
        fp16=not is_bfloat16_supported(),  
        bf16=is_bfloat16_supported(),
        logging_steps=5, 
        optim="paged_adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear", 
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

Map (num_proc=4): 100%|██████████| 40/40 [00:01<00:00, 32.72 examples/s]
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [15]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)


Map: 100%|██████████| 40/40 [00:00<00:00, 1619.91 examples/s]


In [16]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<｜begin▁of▁sentence｜><|im_start|>user\n请根据结尾句补全前面的段落：\n最后一句：The best results were observed in the group that received combinational AAV-NG2Ab+AAV-NT3 treatment..\n<|im_start|>assistant\n完整文本：NG2 is a structurally unique transmembrane chondroitin sulfate proteoglycan (CSPG). Its role in damaged spinal cord is dual. NG2 is considered one of key inhibitory factors restricting axonal growth following spinal injury. Additionally, we have recently detected its novel function as a blocker of axonal conduction. Some studies, however, indicate the importance of NG2 presence in the formation of synaptic contacts. We hypothesized that the optimal treatment would be neutralization of inhibitory functions of NG2 without its physical removal. Acute intraspinal injections of anti-NG2 monoclonal antibodies reportedly prevented an acute block of axonal conduction by exogenous NG2. For prolonged delivery of NG2 function neutralizing antibody, we have developed a novel gene therapy: adeno-associated vec

In [17]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                        \n完整文本：NG2 is a structurally unique transmembrane chondroitin sulfate proteoglycan (CSPG). Its role in damaged spinal cord is dual. NG2 is considered one of key inhibitory factors restricting axonal growth following spinal injury. Additionally, we have recently detected its novel function as a blocker of axonal conduction. Some studies, however, indicate the importance of NG2 presence in the formation of synaptic contacts. We hypothesized that the optimal treatment would be neutralization of inhibitory functions of NG2 without its physical removal. Acute intraspinal injections of anti-NG2 monoclonal antibodies reportedly prevented an acute block of axonal conduction by exogenous NG2. For prolonged delivery of NG2 function neutralizing antibody, we have developed a novel gene therapy: adeno-associated vector (AAV) construct expressing recombinant single-chain variable fragment anti-NG2 antibody (AAV-NG2Ab). We examined effects of 

In [18]:
# 开始训练
trainer_stats = trainer.train()

Step,Training Loss
5,2.765300
10,2.653400
15,2.839900
20,2.756600
25,2.830700
30,2.766900
35,2.843500
40,2.689800
45,1.321500
50,1.214500


In [19]:
trainer.save_model("/scratch/lb4489/LLM/fine_tuned_Qwen")
tokenizer.save_pretrained("/scratch/lb4489/LLM/fine_tuned_Qwen")

('/scratch/lb4489/LLM/fine_tuned_Qwen/tokenizer_config.json',
 '/scratch/lb4489/LLM/fine_tuned_Qwen/special_tokens_map.json',
 '/scratch/lb4489/LLM/fine_tuned_Qwen/tokenizer.json')

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/scratch/lb4489/LLM"

# Import the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("/scratch/lb4489/LLM/fine_tuned_Qwen", cache_dir=model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("/scratch/lb4489/LLM/fine_tuned_Qwen", cache_dir=model_path, trust_remote_code=True).to("cuda")

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.97it/s]


In [27]:

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

user_input = "<|im_start|>user\n请根据结尾句补全前面的段落，并确保内容基于已知神经科学研究，不要编造不存在的术语：\n最后一句：These results highlight the importance of dynorphinergic projections from the central amygdala to the parabrachial nucleus in the regulation of itch signaling.\n<|im_start|>assistant\n完整文本："

inputs = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True, max_length=1024)
inputs = {key: val.to("cuda") for key, val in inputs.items()} 

output_ids = model.generate(
    **inputs,
    max_new_tokens=800,
    temperature=0.5,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.2
)

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

if "<|im_end|>" in generated_text:
    generated_text = generated_text.split("<|im_end|>")[0]

print("生成的段落：", generated_text)


生成的段落： <|im_start|>user
请根据结尾句补全前面的段落，并确保内容基于已知神经科学研究，不要编造不存在的术语：
最后一句：These results highlight the importance of dynorphinergic projections from the central amygdala to the parabrachial nucleus in the regulation of itch signaling.
<|im_start|>assistant
完整文本：The basolateral amygdala (BLA) is important for processing itch and pain signals. In addition to providing sensory information, the BLA plays a key role in the formation of perceptual expectations that influence behavior. The superior colliculus (SC) is involved in the regulation of expectation-driven behaviors, including responses to itch. It has been suggested that the SC receives projections from the BLA that control these processes. However, it is unclear how the number and types of these projections change during different states of immune function or during development. Here we used mice of either gender to test whether the projections from the BLA to the SC are generated through distinct mechanisms depending on their location

In [26]:
print(len(output_ids[0]))  # 查看生成的 token 数量


1089
